In [1]:
#Step 1 Import MAAP
from maap.maap import MAAP
maap = MAAP()

import ipycmc
w = ipycmc.MapCMC()

In [2]:
#Step 1 Cont.
#load some other functions from maap.maap import MAAP
import ipycmc
import sys
import maap
import json
import sys
import shutil
import requests
import os
from os import system
maap = MAAP(maap_host = "api.ops.maap-project.org")

In [3]:
#Step 2 - pull some data over area of interest
#First, run function to return a list of granules toward batch processing
def get_granules_list(granules):
    url_list = []
    output_list = []
    for res in granules:
        url_list.append(res.getDownloadUrl())

    for url in url_list:
        if url[0:5] == 's3://':
            url = url[5:].split('/')
            url[0] += '.s3.amazonaws.com'
            url = 'https://' + '/'.join(url)
        #print(url)
        output_list.append(url)
    return output_list

In [4]:
#Step 2 cont.
#function to download a list of granules locally
def download_granules(granules, output_dir):
    url_list = []
    for res in granules:
        url_list.append(res.getDownloadUrl())

    for url in url_list:
        if url[0:5] == 's3://':
            url = url[5:].split('/')
            url[0] += '.s3.amazonaws.com'
            url = 'https://' + '/'.join(url)
        print(url)
        name = output_dir + url.split('/')[-1]
        print(name)
        r = requests.get(url, allow_redirects=True)
        open(name, 'wb').write(r.content)

In [5]:
#Step 2 cont.
#function to download a single granule
def download_granule(granule, output_dir):
    url = granule.getDownloadUrl()

    if url[0:5] == 's3://':
        url = url[5:].split('/')
        url[0] += '.s3.amazonaws.com'
        url = 'https://' + '/'.join(url)

    name = output_dir + url.split('/')[-1]
    print(name)
    r = requests.get(url, allow_redirects=True)
    open(name, 'wb').write(r.content)

In [6]:
#Step 2 cont.
# Find index for matching ATL03 and ATL08 files from list
def find_matching_granules(atl03_id_list, atl08_id_list):
    id03_list = []
    for i in range (0,len(atl03_id_list)):
        id03_list.append(atl03_id_list[i].split('/')[-1].split('ATL03_')[1].split('_002')[0])

    id08_list = []
    for i in range (0,len(atl08_id_list)):
        id08_list.append(atl08_id_list[i].split('/')[-1].split('ATL08_')[1].split('_003')[0])

    process03 = []
    process08 = []
    count = 0
    no_match_count = 0
    for i in range(0,len(id03_list)):
        count = count + 1
        try:
            matching_08_index = id08_list.index(id03_list[i])
            process03.append(i)
            process08.append(matching_08_index)
        except ValueError:
            no_match_count = no_match_count + 1
            #print('No ATL08 match: ' + id03_list[i])
    print('Total granules: ' + str(count))
    print('Granules without ATL03/ATL08 pair: ' + str(no_match_count))
    
    return process03, process08

In [7]:
# Step 2 cont.
#download the files of interest to local workspace ***if required***

#atl03_granules=maap.searchGranule(collection_concept_id="C1200166513-NASA_MAAP", bounding_box="-180.0,50.0,180.0,90.0", limit=10)
#atl08_granules=maap.searchGranule(collection_concept_id="C1200235747-NASA_MAAP", bounding_box="-180.0,50.0,180.0,90.0", limit=10)

#atl03_granules=maap.searchGranule(collection_concept_id="C1200166513-NASA_MAAP", bounding_box="-102.7684,43.17366,-97.32037,47.48166")
#atl08_granules=maap.searchGranule(collection_concept_id="C1200235747-NASA_MAAP", bounding_box="-102.7684,43.17366,-97.32037,47.48166")

#count=0
#for f in range(0,len(atl03_granules)):
#    ID = atl03_granules[f]['Granule']['OnlineAccessURLs']['OnlineAccessURL']['URL']
#    print(ID)
#    ID2 = atl08_granules[f]['Granule']['OnlineAccessURLs']['OnlineAccessURL']['URL']
#    #print(ID)
#    count=count+1
#    download_dir = '/projects/test_data/'
#    existingfiles = [file for file in os.listdir(download_dir)]
#    if ID in existingfiles or ID2 in existingfiles:
#        print('file exists...')
#    else:   
#        download_granule(atl03_granules[f], download_dir)
#        download_granule(atl08_granules[f], download_dir)
#print(count)

In [24]:
# Step 3
# Search for all granules in AOI (in this case, the entire boreal)
#v002/v003
#atl03_granules=maap.searchGranule(collection_concept_id="C1200166513-NASA_MAAP", bounding_box="-180,50,180,75", limit=1000000)
#atl08_granules=maap.searchGranule(collection_concept_id="C1200235747-NASA_MAAP", bounding_box="-180,50,180,75", limit=1000000)
#v004
atl03_granules=maap.searchGranule(collection_concept_id="C1201300747-NASA_MAAP", bounding_box="-180,50,180,75", limit=1000000)
atl08_granules=maap.searchGranule(collection_concept_id="C1201309827-NASA_MAAP", bounding_box="-180,50,180,75", limit=1000000)


In [25]:
# Step 3 cont
#figure out how to ensure matched pairs
output_list_03 =  get_granules_list(atl03_granules)
output_list_08 =  get_granules_list(atl08_granules)
print(len(output_list_03))

59249


In [26]:
process03, process08 = find_matching_granules(output_list_03, output_list_08)

Total granules: 59249
Granules without ATL03/ATL08 pair: 6333


In [29]:
# Step 3 cont.
# Examine first pair of granules in list
print(output_list_03[process03[0]])
print(output_list_08[process08[0]])
#ATL08_30m_20181014001920_02350103_003_01

https://nasa-maap-data-store.s3.amazonaws.com/file-staging/nasa-map/ATL03___004/2018.10.14/ATL03_20181014001049_02350102_004_01.h5
https://nasa-maap-data-store.s3.amazonaws.com/file-staging/nasa-map/ATL08___004/2018.10.14/ATL08_20181014001049_02350102_004_01.h5


In [30]:
#Step 4
#try to submit a MAAP job on a single file
in_param_dict = {
                         'input_atl03': output_list_03[process03[0]],
                         'input_atl08': output_list_08[process08[0]]}
    
result = maap.submitJob(
    identifier='run_rebinning_ubuntu',
    algo_id='run_rebinning_ubuntu',
    version='master',
    username='lduncanson', # username needs to be the same as whoever created the workspace
    queue='maap-dps-worker-16gb',
    **in_param_dict)
print(result)

{'status': 'failed', 'http_status_code': 500, 'job_id': ''}


In [15]:
#Step 5
#submit many jobs in a loop to process many files, process the first 10 to make sure they're good
for g in range(0,10):
    in_param_dict = {
                             'input_atl03': output_list_03[process03[g]],
                             'input_atl08': output_list_08[process08[g]]}

    result = maap.submitJob(
        identifier='run_rebinning_ubuntu',
        algo_id='run_rebinning_ubuntu',
        version='master',
        username='lduncanson',
        queue='maap-dps-worker-16gb',
        **in_param_dict)
    print(result)

{'status': 'failed', 'http_status_code': 500, 'job_id': ''}
{'status': 'failed', 'http_status_code': 500, 'job_id': ''}
{'status': 'failed', 'http_status_code': 500, 'job_id': ''}
{'status': 'failed', 'http_status_code': 500, 'job_id': ''}
{'status': 'failed', 'http_status_code': 500, 'job_id': ''}
{'status': 'failed', 'http_status_code': 500, 'job_id': ''}
{'status': 'failed', 'http_status_code': 500, 'job_id': ''}
{'status': 'failed', 'http_status_code': 500, 'job_id': ''}
{'status': 'failed', 'http_status_code': 500, 'job_id': ''}
{'status': 'failed', 'http_status_code': 500, 'job_id': ''}


In [21]:
#Step 5 Cont.
# If the first 10 look good, process everything else
for g in range(10,len(process03)):
    in_param_dict = {
                             'input_atl03': output_list_03[process03[g]],
                             'input_atl08': output_list_08[process08[g]]}

    result = maap.submitJob(
        identifier='run_rebinning_ubuntu',
        algo_id='run_rebinning_ubuntu',
        version='master',
        username='lduncanson',
        queue='maap-dps-worker-16gb',
        **in_param_dict)
    print(result)

{'status': 'success', 'http_status_code': 200, 'job_id': 'e7a2456c-9042-4872-b322-71251e125538'}
{'status': 'success', 'http_status_code': 200, 'job_id': 'ee755ca4-2271-4346-ae8a-58a47c6f208c'}
{'status': 'success', 'http_status_code': 200, 'job_id': '5e2b3d18-bf1f-4863-951d-3d808e77cf46'}
{'status': 'success', 'http_status_code': 200, 'job_id': '03e3c483-c36e-4150-9fd9-c515cb6cbe79'}
{'status': 'success', 'http_status_code': 200, 'job_id': '2443c32b-d959-4786-830f-e8b76b7f3fb0'}
{'status': 'success', 'http_status_code': 200, 'job_id': '10fc2cc8-ebbd-4f1d-b792-cb5ee3c221d6'}
{'status': 'success', 'http_status_code': 200, 'job_id': '3ad9c65f-5fbf-41d5-82a8-97b3d414da5c'}
{'status': 'success', 'http_status_code': 200, 'job_id': 'fc2762a0-eb03-4ed0-b63e-f5f93a64e32f'}
{'status': 'success', 'http_status_code': 200, 'job_id': 'a430395b-adc7-4388-a18d-b413039c9153'}
{'status': 'success', 'http_status_code': 200, 'job_id': 'c458fa60-a6a4-46dc-9142-7201f5d160ac'}
{'status': 'success', 'http_st

UnboundLocalError: local variable 'r' referenced before assignment

In [ ]:
#Step 6: analyze results
#move all output csv files from super nested dps_output folder to a folder of interest

#import os, glob
#import shutil
#output_dir = "/projects/r2d2/icesat2_batch_demo/csv_out"
#base_dir = "/projects/r2d2/dps_output/run_icesat2_gabon_ubuntu"
#for file in glob.glob("/projects/r2d2/dps_output/run_icesat2_gabon_ubuntu/master/2020/05/12/**/*.csv", recursive=True):
#    print(file)
#    shutil.copyfile(file, os.path.join(output_dir, os.path.basename(file)))

In [2]:
# Step 7, visualize results of a with a quick plot for a single granule to make sure everything looks alright

import h5py
import numpy as np

atl08 = '/my-private-bucket/dps_output/run_rebinning_ubuntu/master/2021/06/15/18/09/08/386189/ATL08_30m_20181014001049_02350102_003_01.h5'
f = h5py.File(atl08,'r')
x1 = np.asarray(f['gt1l/land_segments/30m_segment/gedi_rh_100'])
x2 = np.asarray(f['gt1l/land_segments/30m_segment/gedi_rh_50'])

import matplotlib.pyplot as plt
plt.plot(x1,'.')
plt.plot(x2,'.')

NameError: name 'h5py' is not defined